In [2]:
# import libraries
import cv2
import mediapipe as mp
import numpy as np

In [3]:
# holistic model
mp_holistic = mp.solutions.holistic

# Initialize mediapipe drawing class
mp_drawing = mp.solutions.drawing_utils

In [4]:
def detection (img, model):
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) # convert the input into RGB format to pass to mediapipe
    results = model.process(img) # detection
    img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR) # convert the input into BGR format to pass to opencv imshow
    return img, results

In [16]:
def draw_landmarks (img, results):
    mp_drawing.draw_landmarks(img, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS, mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1))
    mp_drawing.draw_landmarks(img, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, mp_drawing.DrawingSpec(color=(255,128,0), thickness=2, circle_radius=2),mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=2))
    mp_drawing.draw_landmarks(img, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, mp_drawing.DrawingSpec(color=(255,51,153), thickness=2, circle_radius=2),mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=2))
    mp_drawing.draw_landmarks(img, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, mp_drawing.DrawingSpec(color=(255,51,153), thickness=2, circle_radius=2),mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=2))

In [17]:
# get webcam
cap = cv2.VideoCapture('./videoplayback.mp4')
#cap = cv2.VideoCapture(0)

with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while True:
        ret, frame = cap.read()

        img, results = detection(frame, holistic) # make detection
        draw_landmarks(img,results) # draw landmarks

        cv2.imshow('Human Pose Estimation', img)
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

In [28]:
len(results.pose_landmarks.landmark)

33

In [29]:
if results.pose_landmarks.landmark:
    for i in range(4):
        print(f'{mp_holistic.PoseLandmark(i).name}:\n{results.pose_landmarks.landmark[mp_holistic.PoseLandmark(i).value]}') 

NOSE:
x: 0.4362970292568207
y: 0.2314050942659378
z: -0.38031822443008423
visibility: 0.9999972581863403

LEFT_EYE_INNER:
x: 0.44820255041122437
y: 0.22117887437343597
z: -0.3325987160205841
visibility: 0.99998939037323

LEFT_EYE:
x: 0.4575251638889313
y: 0.22145380079746246
z: -0.33260613679885864
visibility: 0.9999900460243225

LEFT_EYE_OUTER:
x: 0.46521201729774475
y: 0.22194400429725647
z: -0.33270493149757385
visibility: 0.9999896287918091

